In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from zipline import run_algorithm
from zipline.api import order_target_percent, record, symbol
from datetime import datetime
import pytz

%matplotlib inline 

In [2]:
def initialize(context):
    dji = ['APPL',
          'AXP',
          'BA',
          'CAT',
          'CSCO',
          'CVX',
          'DIS',
          'DWDP', # ? "DD" or "GE"
          'GS',
          'HD',
          'IBM',
          'INTC',
          'JNJ',
          'JPM',
          'KO',
          'MCD',
          'MMM',
          'MRK',
          'MSFT',
          'NKE',
          'PFE',
          'PG',
          'TRV',
          'UNH',
          'UTX', #RTX
          'V',
          'VZ',
          'WBA',
          'WMT',
          'XOM']
    
    # Make a list of symbols from the list of tickers
    context.dji_symbols = [symbol(s) for s in dji]
    
    # Moving average window
    context.index_average_window = 100

def handle_data(data, context):
    # Get history of all the stock 
    stock_hist = data.history(context.dji_symbols, 'close', context.index_average_window, '1d')
    
    # Make empty DataFrame to start with
    stock_analytics = pd.DateFrame()
    
    # Add column for above or below average
    stock_analytics['above_mean'] = stock_hist.iloc[-1] > stock_hist.mean()
    
    # Set weights for stocks to buy
    stock_analytics.loc[stock_analytics['above_mean'] == True, 'weight'] = 1/len(context.dji_symbols)
    
    # Set zero weights for the rest
    stock_analytics.loc[stock_analytics['above_mean'] == False, 'weight'] = 0.0
    
    # Iterate each row and place trades
    for stock, analytics in stock_analytics.itterrows():
        # Check if the stock can be traded?
        if data.can_trade(stock):
            # Place the trade
            order_target_percent(stock, analytics['weight'])
            
def analyze(context, perf):
    fig = plt.figure(figsize=(12, 8))
    
    # First chart
    ax1 = fig.add_subplot(311)
    ax1.set_title('Strategy results')
    ax1.plot(pref['portfolio_value'], linestyle='-', label='Equity Curve', linewidth=3.0)
    ax1.legend()
    ax1.grid(False)
    
    # Second chart
    ax2 = fig.add_subplot(312)
    ax2.plot(perf['gross_leverage'], linestule='-', linewidth=1.0)
    ax2.legend()
    ax2.grid(True)
    
    # Third chart
    ax3 = fig.add_subplot(313)
    ax3.plot(perf['returns'], label='Returns', linestyle='-.', linewidth=1.0)
    ax3.legend()
    ax3.grid(True)
    
    

In [3]:
# Set start and end dates
start = datetime(2003, 1, 1, tzinfo=pytz.UTC)
end = datetime(2007, 12, 31, tzinfo=pytz.UTC)

In [4]:
# Run the backtest and get the results
results = run_algorithm(start=start, 
                        end=end,
                        initialize=initialize,
                        analyze=analyze, 
                        handle_data=handle_data,
                        capital_base=10000,
                        data_frequency='daily', 
                        bundle='quandl'
                       )

AssertionError: 